In [1]:
import sys
sys.path.append("../")
from ccfm.ccfm import (
    load_cfm_traces, 
    make_3d_fault_mesh, 
    load_nshm_traces, 
    load_canada_traces,
    make_tri_mesh,
    write_cfm_tri_meshes,
    write_cfm_trace_geojson,
)

In [2]:
from ccfm.geom import (
    sample_polyline,
    sample_polyline_to_n_pts,
    add_fixed_elev_to_trace,
    _draw_pt_profile,
    get_contours_from_profiles,
    haversine_distance,
    _straight_profile_n_pts,
    EARTH_RAD_KM,
)

In [3]:
from ccfm.cfm_io import make_3d_tri_multipolygon, write_json

In [4]:
import json
import numpy as np
import matplotlib.pyplot as plt

In [5]:
trace_file = '../misc_data/test_data/3d_test_faults.geojson'

with open(trace_file) as f:
    trace_gj = json.load(f)
    
    traces = {}
    for f in trace_gj['features']:
        traces[int(f['properties']['id'])] = f

In [6]:
# contours from the Manastash fault
man_contours = [traces[0], traces[1], traces[4]]

In [7]:
def prepare_fault_contours(fault_contours, **kwargs):
    fc_sorted = sorted(fault_contours, key=lambda x: x['properties']['level'])
    trace = fc_sorted[0]
    trace_sampled = sample_polyline(trace['geometry']['coordinates'], **kwargs)
    trace_sampled = add_fixed_elev_to_trace(trace_sampled, trace['properties']['depth'])
    
    contours_out = []
    contours_out.append(trace_sampled)
    
    n_trace_pts = len(trace_sampled)
    for trace in fc_sorted[1:]:
        trace_sampled = sample_polyline_to_n_pts(trace['geometry']['coordinates'], n_trace_pts)
        trace_sampled = add_fixed_elev_to_trace(trace_sampled, trace['properties']['depth'])
        contours_out.append(trace_sampled)
    
    
    return contours_out

In [8]:
prepped_contours = prepare_fault_contours(man_contours, pt_distance=2.0)

In [9]:
def make_mesh_from_prepared_contours(contours, vert_pt_spacing=None, down_dip_pt_spacing=None):
    #if (vert_pt_spacing is None) and (down_dip_pt_spacing is None):
    #    raise ValueError("Need vert_pt_spacing or down_dip_pt_spacing")
    #elif (vert_pt_spacing is not None) and (down_dip_pt_spacing is not None):
    #    raise ValueError("Choose vert_pt_spacing or down_dip_pt_spacing")
    
    num_contour_sets = len(contours) - 1
    
    all_contours = []
    
    for i_cs in range(num_contour_sets):
        top_contour_elev = contours[i_cs][0][2]
        bottom_contour_elev = contours[i_cs+1][0][2]
        vert_distance = (top_contour_elev - bottom_contour_elev) / 1000.
        
        # get horizontal distance between points
        # using first point
        hor_distance = haversine_distance(contours[i_cs][0][0],
                                          contours[i_cs][0][1],
                                          contours[i_cs+1][0][0],
                                          contours[i_cs+1][0][1],
                                         )
        
        down_dip_distance = np.sqrt(vert_distance**2 + hor_distance**2)
        
        n_pts = int(round( down_dip_distance / down_dip_pt_spacing)) + 1    
          
        profiles = [_draw_pt_profile(contours[i_cs][j],
                                     contours[i_cs+1][j],
                                     n_pts)
                    for j in range(len(contours[i_cs]))]
        
        if i_cs == 0:
            return_top = True
        else:
            return_top = False
        
        contour_set = get_contours_from_profiles(profiles,
                                                return_top,
                                                )
        all_contours.extend(contour_set)
        
    return all_contours
    

In [10]:
profs = make_mesh_from_prepared_contours(prepped_contours, 
                                         down_dip_pt_spacing=2.0)

In [11]:
def make_3d_geojson_from_contours(fault_contours, #fault=None,
                                  horiz_pt_spacing=2.0,
                                  vert_pt_spacing=None, 
                                  down_dip_pt_spacing=None):
    prepped_contours = prepare_fault_contours(fault_contours, 
                                              pt_distance=horiz_pt_spacing)
    mesh = make_mesh_from_prepared_contours(prepped_contours,
                                            down_dip_pt_spacing=down_dip_pt_spacing)
    tri_mesh = make_tri_mesh(mesh)

    return tri_mesh

In [12]:
fgj = make_3d_geojson_from_contours(man_contours, 
                                    horiz_pt_spacing=2.0,
                                    down_dip_pt_spacing=2.0)

In [13]:
write_cfm_tri_meshes("../misc_data/test_data/3d_test_meshes.geojson",
                     [fgj],
                     [{'properties':{'name': 'manastash_3d'}}],)